# Fase 2 – Classificação por domínio/forno/granularidade
Atividade 2.3: enriquecer o inventário com colunas de domínio, forno e granularidade.

## Escopo
- carregar o inventário e o manifest intermediário;
- normalizar `pasta_raiz` → `pasta_raiz_canonica`;
- derivar `dominio`, `forno`, `granularidade` e colunas de observações;
- persistir `manifest_arquivos_classificados.csv`;
- exibir resumos e amostras por pasta.

In [1]:
from __future__ import annotations
from pathlib import Path
import unicodedata
import re
import pandas as pd


In [2]:
def find_fase1_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if candidate.name == 'fase_1_diagnostico':
            return candidate
    raise RuntimeError('Não encontrei a pasta fase_1_diagnostico a partir deste notebook')

CURRENT_DIR = Path.cwd().resolve()
FASE1_ROOT = find_fase1_root(CURRENT_DIR)
INVENTARIO_PATH = FASE1_ROOT / 'dados' / 'manifestos_inicial' / 'inventario_dados_iniciais.csv'
MANIFEST_INTER_PATH = FASE1_ROOT / 'dados' / 'manifestos_inicial' / 'manifest_intermediario.csv'
OUTPUT_PATH = FASE1_ROOT / 'dados' / 'manifestos_final' / 'manifest_arquivos_classificados.csv'

FASE1_ROOT, INVENTARIO_PATH.exists(), MANIFEST_INTER_PATH.exists(), OUTPUT_PATH.parent.exists()

(PosixPath('/home/wilson/Maringa/fase_1_diagnostico'), True, True, True)

In [3]:
inventario_df = pd.read_csv(INVENTARIO_PATH)
manifest_inter_df = pd.read_csv(MANIFEST_INTER_PATH)
len(inventario_df), len(manifest_inter_df)

(145, 167)

In [4]:
CANONICAL_MAP = {
    'consumo fornos': 'Consumo Fornos',
    'corridas': 'Corridas',
    'dicionario': 'Dicionario',
    'dicionario ': 'Dicionario',
    'eletrodo': 'Eletrodo',
    'informacoes diaria': 'Informacoes Diaria',
    'informacoes diarias': 'Informacoes Diaria',
    'informacoes': 'Informacoes Diaria',
    'supervisorio forno 4': 'Supervisorio Forno 4',
    'supervisorio forno 5': 'Supervisorio Forno 5',
}

def strip_accents(value: str) -> str:
    if not isinstance(value, str):
        return ''
    normalized = unicodedata.normalize('NFKD', value)
    without_accents = ''.join(ch for ch in normalized if not unicodedata.combining(ch))
    without_nbsp = without_accents.replace('\u00a0', ' ')
    collapsed = re.sub(r'\s+', ' ', without_nbsp).strip()
    return collapsed

def canonicalize_pasta(value: str) -> str:
    base = strip_accents(value).lower()
    canonical = CANONICAL_MAP.get(base)
    if canonical:
        return canonical
    # tentar remover pluralizacao simples
    base_simple = base.replace('ã', 'a')
    if base_simple in CANONICAL_MAP:
        return CANONICAL_MAP[base_simple]
    return 'Desconhecido'

inventario_df['pasta_raiz_canonica'] = inventario_df['pasta_raiz'].apply(canonicalize_pasta)
manifest_inter_df['pasta_raiz_canonica'] = manifest_inter_df['pasta_raiz'].apply(canonicalize_pasta)
inventario_df[['pasta_raiz', 'pasta_raiz_canonica']].drop_duplicates().sort_values('pasta_raiz_canonica').head(10)

,pasta_raiz,pasta_raiz_canonica
0,Consumo Fornos,Consumo Fornos
39,Corridas,Corridas
79,Dicionário,Dicionario
100,Eletrodo,Eletrodo
101,Informações Diária,Informacoes Diaria
141,Supervisorio Forno 4,Supervisorio Forno 4
143,Supervisorio Forno 5,Supervisorio Forno 5


In [5]:
DOMINIO_MAP = {
    'Consumo Fornos': 'consumo_fornos',
    'Corridas': 'corridas',
    'Dicionario': 'dicionario',
    'Eletrodo': 'eletrodo',
    'Informacoes Diaria': 'informacoes_diarias',
    'Supervisorio Forno 4': 'supervisorio',
    'Supervisorio Forno 5': 'supervisorio',
}

def assign_dominio(row):
    dominio = DOMINIO_MAP.get(row['pasta_raiz_canonica'])
    if dominio:
        return dominio, None
    return 'desconhecido', f"Sem mapeamento para {row['pasta_raiz']}"

dominio_data = inventario_df.apply(assign_dominio, axis=1, result_type='expand')
inventario_df['dominio'] = dominio_data[0]
inventario_df['observacoes_dominio'] = dominio_data[1]
inventario_df['dominio'].value_counts()

dominio
corridas               40
informacoes_diarias    40
consumo_fornos         39
dicionario             21
supervisorio            4
eletrodo                1
Name: count, dtype: int64

In [6]:
def detect_forno(nome_arquivo: str) -> tuple[str, str | None]:
    nome = (nome_arquivo or '').upper()
    matches = re.findall(r'F([1-5])', nome)
    unique = sorted(set(matches))
    if len(unique) == 1:
        return f"F{unique[0]}", None
    if len(unique) > 1:
        return 'todos', f"Arquivo referencia multiplos fornos: {','.join(f'F{x}' for x in unique)}"
    if 'TODOS' in nome or 'ALL' in nome:
        return 'todos', None
    return 'nao_definido', 'Sem indicacao de forno no nome_arquivo'

def assign_forno(row):
    canonical = row['pasta_raiz_canonica']
    if canonical == 'Supervisorio Forno 4':
        return 'F4', None
    if canonical == 'Supervisorio Forno 5':
        return 'F5', None
    return detect_forno(row['nome_arquivo'])

forno_data = inventario_df.apply(assign_forno, axis=1, result_type='expand')
inventario_df['forno'] = forno_data[0]
inventario_df['observacoes_forno'] = forno_data[1]
inventario_df['forno'].value_counts()

forno
F5              32
F4              29
F2              24
F3              24
F1              23
nao_definido    13
Name: count, dtype: int64

In [7]:
def infer_granularidade(row):
    dominio = row['dominio']
    nome = (row['nome_arquivo'] or '').lower()
    if dominio == 'supervisorio':
        return 'minuto', None
    if dominio == 'corridas':
        return 'corrida', None
    if dominio == 'consumo_fornos':
        if 'diario' in nome or re.search(r'\bdia', nome):
            return 'dia', None
        if 'hora' in nome:
            return 'hora', None
        return 'nao_definida', 'Sem indicacao clara para consumo_fornos'
    if dominio == 'dicionario':
        return 'meta', None
    if dominio == 'informacoes_diarias':
        return 'dia', None
    if dominio == 'eletrodo':
        if 'corrida' in nome:
            return 'corrida', None
        if 'dia' in nome:
            return 'dia', None
        return 'nao_definida', 'Sem indicacao clara para eletrodo'
    return 'nao_definida', 'Dominio sem regra especifica'

granularidade_data = inventario_df.apply(infer_granularidade, axis=1, result_type='expand')
inventario_df['granularidade'] = granularidade_data[0]
inventario_df['observacoes_granularidade'] = granularidade_data[1]
inventario_df['granularidade'].value_counts()

granularidade
nao_definida    40
corrida         40
dia             40
meta            21
minuto           4
Name: count, dtype: int64

In [8]:
manifest_summary = (
    manifest_inter_df
    .groupby('pasta_raiz_canonica')
    .agg(
        fontes_manifesto=('fonte_manifesto', lambda x: ','.join(sorted(set(x)))),
        qtd_arquivos_reportada_manifesto=('qtd_arquivos_reportada', 'sum'),
        descricao_manifesto=('descricao_manifesto', lambda x: '; '.join(sorted(set(filter(None, x)))))
    )
    .reset_index()
 )

enriched_df = inventario_df.merge(
    manifest_summary,
    on='pasta_raiz_canonica',
    how='left'
 )
enriched_df.head(3)

,path_local,pasta_raiz,nome_arquivo,extensao,tamanho_bytes,data_modificacao,hash_sha256,pasta_raiz_canonica,dominio,observacoes_dominio,forno,observacoes_forno,granularidade,observacoes_granularidade,fontes_manifesto,qtd_arquivos_reportada_manifesto,descricao_manifesto
0,dados/dados_iniciais/Consumo Fornos/2018_F2_Co...,Consumo Fornos,2018_F2_Consumo.csv,.csv,28950526,2025-04-29T15:22:50+00:00,7d942c3e034dca9818f3cda2f5a5ab173b370087e66acd...,Consumo Fornos,consumo_fornos,None,F2,None,nao_definida,Sem indicacao clara para consumo_fornos,"index_resumo_por_pasta,manifest_index",78,"2018_CONSUMO; F5, F4, F3, F2; CSV; 2019_CONSUM..."
1,dados/dados_iniciais/Consumo Fornos/2018_F3_Co...,Consumo Fornos,2018_F3_Consumo.csv,.csv,29621377,2025-04-29T15:23:02+00:00,9f93ce3cfbd1885b6aaea9ad42ac9df4c296996ae239a2...,Consumo Fornos,consumo_fornos,None,F3,None,nao_definida,Sem indicacao clara para consumo_fornos,"index_resumo_por_pasta,manifest_index",78,"2018_CONSUMO; F5, F4, F3, F2; CSV; 2019_CONSUM..."
2,dados/dados_iniciais/Consumo Fornos/2018_F4_Co...,Consumo Fornos,2018_F4_Consumo.csv,.csv,30486644,2025-04-29T15:23:16+00:00,8ee5c4e41d1f732906f78c17f624413a2412aba3f3ddcb...,Consumo Fornos,consumo_fornos,None,F4,None,nao_definida,Sem indicacao clara para consumo_fornos,"index_resumo_por_pasta,manifest_index",78,"2018_CONSUMO; F5, F4, F3, F2; CSV; 2019_CONSUM..."


In [9]:
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
enriched_df.to_csv(OUTPUT_PATH, index=False)
OUTPUT_PATH

PosixPath('/home/wilson/Maringa/fase_1_diagnostico/dados/manifestos_final/manifest_arquivos_classificados.csv')

In [10]:
resumo_dominio = enriched_df.groupby('dominio').size().sort_values(ascending=False)
resumo_forno = enriched_df.groupby('forno').size().sort_values(ascending=False)
resumo_granularidade = enriched_df.groupby('granularidade').size().sort_values(ascending=False)
resumo_dominio, resumo_forno, resumo_granularidade

(dominio
 corridas               40
 informacoes_diarias    40
 consumo_fornos         39
 dicionario             21
 supervisorio            4
 eletrodo                1
 dtype: int64,
 forno
 F5              32
 F4              29
 F2              24
 F3              24
 F1              23
 nao_definido    13
 dtype: int64,
 granularidade
 corrida         40
 dia             40
 nao_definida    40
 meta            21
 minuto           4
 dtype: int64)

In [11]:
samples_por_pasta = (
    enriched_df
    .groupby('pasta_raiz_canonica', group_keys=False)
    .head(2)
    .loc[:, [
        'pasta_raiz',
        'pasta_raiz_canonica',
        'nome_arquivo',
        'dominio',
        'forno',
        'granularidade',
        'observacoes_dominio',
        'observacoes_forno',
        'observacoes_granularidade'
    ]]
 )
samples_por_pasta

,pasta_raiz,pasta_raiz_canonica,nome_arquivo,dominio,forno,granularidade,observacoes_dominio,observacoes_forno,observacoes_granularidade
0,Consumo Fornos,Consumo Fornos,2018_F2_Consumo.csv,consumo_fornos,F2,nao_definida,None,None,Sem indicacao clara para consumo_fornos
1,Consumo Fornos,Consumo Fornos,2018_F3_Consumo.csv,consumo_fornos,F3,nao_definida,None,None,Sem indicacao clara para consumo_fornos
39,Corridas,Corridas,2018_F1_Corrida.csv,corridas,F1,corrida,None,None,None
40,Corridas,Corridas,2018_F2_Corrida.csv,corridas,F2,corrida,None,None,None
79,Dicionário,Dicionario,Dic_Consumo.pdf,dicionario,nao_definido,meta,None,Sem indicacao de forno no nome_arquivo,None
80,Dicionário,Dicionario,Dic_Consumo.txt,dicionario,nao_definido,meta,None,Sem indicacao de forno no nome_arquivo,None
100,Eletrodo,Eletrodo,Medição Eletrodo.csv,eletrodo,nao_definido,nao_definida,None,Sem indicacao de forno no nome_arquivo,Sem indicacao clara para eletrodo
101,Informações Diária,Informacoes Diaria,2018_F1_Inf.Diario.csv,informacoes_diarias,F1,dia,None,None,None
102,Informações Diária,Informacoes Diaria,2018_F2_Inf.Diario.csv,informacoes_diarias,F2,dia,None,None,None
141,Supervisorio Forno 4,Supervisorio Forno 4,F4_2024_1S.csv,supervisorio,F4,minuto,None,None,None
